In [1]:

"""
Serial GPU matrix factorization using CuPy .
After training,enter a user ID to get top-N recommendations.
"""
import cupy as cp
import pandas as pd

ratings_path = r'C:\Python ML programs\cu2rec\Parallel-matrix-factorization-with-cuda-for-recommended-systems\ratings.csv'
factors      = 50
lr           = 0.005
reg          = 0.02
epochs       = 5
batch_size   = 4096
top_n        = 10

# 1) Load data on CPU then send to GPU
df      = pd.read_csv(ratings_path, usecols=["userId","movieId","rating"])
users   = cp.array(df.userId.values.astype(int))
items   = cp.array(df.movieId.values.astype(int))
ratings = cp.array(df.rating.values.astype(float))

n_users = int(users.max()) + 1
n_items = int(items.max()) + 1
N       = ratings.size
print(f"Loaded {N} ratings from {n_users} users and {n_items} items.")

# 2) Initialize U, V on GPU
cp.random.seed(0)
U = cp.random.randn(n_users, factors, dtype=cp.float32) * 0.01
V = cp.random.randn(n_items, factors, dtype=cp.float32) * 0.01

idx_all = cp.arange(N, dtype=cp.int32)

# 3) Training loop
for epoch in range(1, epochs+1):
    perm = cp.random.permutation(idx_all)
    for start in range(0, N, batch_size):
        b  = perm[start:start+batch_size]
        ub = users[b]; ib = items[b]; rb = ratings[b]
        preds = cp.sum(U[ub] * V[ib], axis=1)
        errs  = preds - rb
        U[ub] -= lr * (errs[:, None] * V[ib] + reg * U[ub])
        V[ib] -= lr * (errs[:, None] * U[ub] + reg * V[ib])
    mse = cp.float64(0.0)
    for start in range(0, N, batch_size):
        sl = slice(start, start+batch_size)
        chunk = cp.sum((cp.sum(U[users[sl]] * V[items[sl]], axis=1) - ratings[sl])**2)
        mse += chunk
    rmse = cp.sqrt(mse / N)
    print(f"Epoch {epoch}/{epochs} RMSE: {float(rmse):.4f}")

print("Training complete.")

Loaded 32000204 ratings from 200949 users and 292758 items.
Epoch 1/5 RMSE: 1.0452
Epoch 2/5 RMSE: 0.9386
Epoch 3/5 RMSE: 0.8928
Epoch 4/5 RMSE: 0.8629
Epoch 5/5 RMSE: 0.8411
Training complete.


In [2]:
# 4) Prompt for user ID and recommendation
uid_str = input(f"\nEnter a user ID (0-{n_users-1}) for recommendations, or blank to exit: ").strip()
if uid_str:
    try:
        uid = int(uid_str)
        if 0 <= uid < n_users:
            rated = items[users == uid]
            scores = V.dot(U[uid])
            scores[rated] = -cp.inf
            top_items = cp.argsort(scores)[-top_n:][::-1]
            print(f"\nTop {top_n} recommendations for user {uid}:")
            for item in top_items.get():
                print(f"  Item ID {int(item)}, Score {float(scores[item]):.4f}")
        else:
            print("User ID out of range.")
    except ValueError:
        print("Invalid input; please enter an integer.")
else:
    print("No user ID entered. Exiting.")


Top 10 recommendations for user 1125:
  Item ID 159817, Score 4.5637
  Item ID 171011, Score 4.5152
  Item ID 170705, Score 4.5139
  Item ID 220528, Score 4.4721
  Item ID 105250, Score 4.4705
  Item ID 858, Score 4.4655
  Item ID 1203, Score 4.4259
  Item ID 318, Score 4.4241
  Item ID 171495, Score 4.4214
  Item ID 93040, Score 4.4206
